In [ ]:
import pandas as pd
import streamlit as st
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor


cidades = pd.read_excel('BRCidadesRegiao.xlsx')


cidades = cidades[['regiao_imediata', 'cidade', 'estado', 'longitude', 'latitude', 'altitude',
'area_territorial', 'populacao_estimada', 'densidade_demografica', 'pib_per_capita', 'receitas_realizadas',
'despesas_empenhadas', 'saldo_receitas_despesas', 'unidades', 'unidades_urbanas', 'unidades_rurais', 'area_plantada', 'producao_graos',
'IDHM_Renda', 'IDHM_Longevidade', 'IDHM_Educacao', 'GVA_agropecuaria', 'GVA_industria', 'GVA_Servicos', 'GVA_publico',
'GDP', 'GDP_populacao', 'GDP_per_capita', 'estabelecimentos', 'carros', 'motos'
]].copy()


cidades.info()


cidades.replace([np.inf, -np.inf], np.nan, inplace=True)
cidades.dropna(inplace=True)


cidades['estado'].unique()


estados_desejados = ['BA', 'ES', 'DF']
cidades_especificas = cidades[cidades['estado'].isin(estados_desejados)].copy()
st.write("Cidades de BA, ES e DF:", cidades_especificas)


vif_dt = cidades_especificas[[
'area_territorial', 'populacao_estimada', 'densidade_demografica', 'pib_per_capita', 'receitas_realizadas',
'despesas_empenhadas', 'saldo_receitas_despesas', 'unidades', 'unidades_urbanas', 'unidades_rurais', 'area_plantada', 'producao_graos',
'IDHM_Renda', 'IDHM_Longevidade', 'IDHM_Educacao', 'GVA_agropecuaria', 'GVA_industria', 'GVA_Servicos', 'GVA_publico',
'GDP', 'GDP_populacao', 'GDP_per_capita', 'estabelecimentos', 'carros', 'motos'
]]


vif_data = pd.DataFrame()
vif_data['variavel'] = vif_dt.columns
vif_data['VIF'] = [variance_inflation_factor(vif_dt.values, i)
    for i in range(len(vif_dt.columns))]
round(vif_data,4)


vif_dt = cidades_especificas[[
'area_territorial', 'populacao_estimada', 'pib_per_capita',
'area_plantada', 'IDHM_Educacao', 'GDP']]


vif_data = pd.DataFrame()
vif_data['variavel'] = vif_dt.columns
vif_data['VIF'] = [variance_inflation_factor(vif_dt.values, i)
    for i in range(len(vif_dt.columns))]
st.write("Variáveis VIF: ", vif_data)


vif_dt = vif_dt.copy()
cortes = pd.DataFrame(columns=['Coluna', 'Out_max', 'Out_min'])
for coluna in vif_dt.columns:
    print(coluna)
    out_max = vif_dt[coluna].quantile(0.75) + ((vif_dt[coluna].quantile(0.75) - vif_dt[coluna].quantile(0.25)) * 1.5)
    print(out_max)
    vif_dt[coluna] = vif_dt[coluna].apply(lambda x : out_max if x > out_max else x)
    out_min = vif_dt[coluna].quantile(0.25) - ((vif_dt[coluna].quantile(0.75) - vif_dt[coluna].quantile(0.25)) * 1.5)
    print(out_min)
    vif_dt[coluna] = vif_dt[coluna].apply(lambda x : out_min if x < out_min else x)
    cortes = cortes.append({'Coluna': coluna, 'Out_max': out_max, 'Out_min': out_min}, ignore_index=True)


st.write("Cortes realizados para os outliers:", cortes)


# mostra o dataframe com os dados após os cortes
st.write("Dados após os cortes:", vif_dt)


colunas = ['area_territorial', 'populacao_estimada', 'pib_per_capita', 'area_plantada', 'IDHM_Educacao', 'GDP']
sel = colunas
cidadesz = vif_dt[sel].copy()
listaz = []
for coluna in colunas:
    if coluna != "cidade":
        print(coluna)
        listaz.insert(0, 'z_'+coluna)
        media = cidadesz[coluna].mean()
        dp = cidadesz[coluna].std()
        cidadesz['z_'+coluna] = cidadesz[coluna].apply(lambda x : (x - media) / dp)
cidadesz['totalz'] = cidadesz[listaz].sum(axis=1)
cidadesz['ranking'] = cidadesz['totalz'].rank(ascending=False)


st.write("z-score:", cidadesz)


cidades_especificas = cidades_especificas.merge(cidadesz, right_index=True, left_index=True, how = 'left')


st.write("10 melhores cidades (BA):", cidades_especificas[cidades_especificas['estado'] == 'BA'].sort_values(by='ranking')[0:10])
st.write("10 melhores cidades (ES):", cidades_especificas[cidades_especificas['estado'] == 'ES'].sort_values(by='ranking')[0:10])
st.write("10 melhores cidades (DF):", cidades_especificas[cidades_especificas['estado'] == 'DF'].sort_values(by='ranking')[0:10])


st.write("10 piores cidades cidades (BA):", cidades_especificas[cidades_especificas['estado'] == 'BA'].sort_values(by='ranking', ascending=False)[0:10])
st.write("10 piores cidades cidades (ES):", cidades_especificas[cidades_especificas['estado'] == 'ES'].sort_values(by='ranking', ascending=False)[0:10])
st.write("10 piores cidades cidades (DF):", cidades_especificas[cidades_especificas['estado'] == 'DF'].sort_values(by='ranking', ascending=False)[0:10])


regioes = cidades.groupby(['estado','regiao_imediata'])[['area_territorial', 'populacao_estimada', 'pib_per_capita', 'area_plantada', 'IDHM_Educacao', 'GDP']].mean().reset_index().copy()


regioes_especificas = regioes[regioes['estado'].isin(estados_desejados)]


colunas = ['area_territorial', 'populacao_estimada', 'pib_per_capita', 'area_plantada', 'IDHM_Educacao', 'GDP']
sel = colunas
regioesz = regioes_especificas[sel].copy()
listaz = []
for coluna in colunas:
    if (coluna != "estado") & (coluna != "regiao_imediata"):
        print(coluna)
        listaz.insert(0, 'z_'+coluna)
        media = regioesz[coluna].mean()
        dp = regioesz[coluna].std()
        regioesz['z_'+coluna] = regioesz[coluna].apply(lambda x : (x - media) / dp)
regioesz['totalz'] = regioesz[listaz].sum(axis=1)
regioesz['ranking'] = regioesz['totalz'].rank(ascending=False)


regioes_especificas = regioes_especificas.merge(regioesz, right_index=True, left_index=True, how = 'left')


st.write("3 melhores regiões (BA):", regioes_especificas[regioes_especificas['estado'] == 'BA'].sort_values(by='ranking')[0:3])
st.write("3 melhores regiões (ES):", regioes_especificas[regioes_especificas['estado'] == 'ES'].sort_values(by='ranking')[0:3])
st.write("3 melhores regiões (DF):", regioes_especificas[regioes_especificas['estado'] == 'DF'].sort_values(by='ranking')[0:3])


st.write("3 piores regiões (BA):", regioes_especificas[regioes_especificas['estado'] == 'BA'].sort_values(by='ranking', ascending=False)[0:3])
st.write("3 piores regiões (ES):", regioes_especificas[regioes_especificas['estado'] == 'ES'].sort_values(by='ranking', ascending=False)[0:3])
st.write("3 piores regiões (DF):", regioes_especificas[regioes_especificas['estado'] == 'DF'].sort_values(by='ranking', ascending=False)[0:3])